# Paramiko demo Jupyter Notebook

## Import Modules

In [1]:
import paramiko
from getpass import getpass
from pathlib import Path

The funny thing is that I was just trying to set up a Debian VM to build a [buildroot](https://buildroot.org/)
system, and instantly forgot the root password, before setting up any accounts. I knew it was a gibberish
acronym of a specific 6-letter word and I knew the first letter, so I tried to brute-force it with paramiko.

*(*<split>***NOTE:*** *My first set-up step for newly provisioned Linux VMs is to run an ansible playbook that,
among other things, disables logging in as root, and creates an account called `eliminmax` which can only be
accessed with my SSH key, and they are only accessible to my laptop, so I don't put any effort into password
security for them. If it were not a single-use password in an isolated environment, I'd take it far,* ***far***
*more seriously.)*

## Try to authenticate

I set up a minimal Rocky Linux VMware Workstation VM on my laptop, with an SSH server, specifically for this
assignment, with the hostname `paramiko-host`. My laptop runs the `dnsmasq` DNS/DHCP server, accessible to VMs
and to localhost, which provides DNS resolution for my VMs, and from my laptop to my VMs.

In [2]:
user = 'eliminmax'
host = 'paramiko-host'
port = 22

# Allow up to 3 password attempts
for _ in range(3):
    try:
        password = getpass(prompt=f'Password for {user}@{host}:{port+":" if port != 22 else ""} ')
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect(host, port, user, password)
        break
    except paramiko.AuthenticationError:
        print(f'Authentication Failed for {user}@{host}{port+":" if port != 22 else ""}')
else:
    # If authentication failed, we should stop
    exit()

Password for eliminmax@paramiko-host: ········


### Run a command

In [4]:
commands = ['ps -ef', 'netstat -an --inet', 'last -adx', 'cut -d: -f1 /etc/passwd']

for command in commands:
    # save the command i/o streams
    stdin, stdout, stderr = ssh.exec_command(command)

    sep_header = f"### BEGIN {command} ###\n\n"
    sep_footer = f"#### END {command} ####\n\n"

    '''
    # Get results from stdout
    lines = stdout.readlines()
    # convert the list into a strings
    output = ''.join(lines)
    '''
    # This is a simpler way to get the same result
    # print(stdout.read())
    cmd_output = sep_header + stdout.read().decode() + sep_footer # decode bytes to string

# Save to file
    with open('ssh_results', 'a') as f:
        f.write(cmd_output)

## SFTP Time, baby!

In [ ]:
sftp = ssh.open_sftp()
files = ['/etc/hosts', '/etc/passwd', '/var/log/auth.log']
local_dir = Path.cwd().joinpath('downloads', host)
# ensure that local_dir exists:
if not local_dir.is_dir():
    local_dir.mkdir(parents=True)

for file in files:
    sftp.get(file, local_dir.joinpath(Path.name(file)))

### Close the SSH connection

In [5]:
ssh.close()